### Basic SMA crossover strategy

In [5]:
import backtrader as bt
import pandas as pd
import ccxt
from datetime import datetime

class SmaCross(bt.Strategy):
    params = dict(
        pfast=10,  
        pslow=30,  
        risk_perc=0.95  
    )

    def __init__(self):
        self.sma1 = bt.ind.SMA(self.data.close, period=self.p.pfast)
        self.sma2 = bt.ind.SMA(self.data.close, period=self.p.pslow)
        self.crossover = bt.ind.CrossOver(self.sma1, self.sma2)
        
        # For manual drawdown calculation
        self.max_value = self.broker.getvalue()
        self.max_drawdown = 0

    def next(self):
        # Update drawdown
        current_value = self.broker.getvalue()
        self.max_value = max(self.max_value, current_value)
        drawdown = (self.max_value - current_value) / self.max_value
        self.max_drawdown = max(self.max_drawdown, drawdown)

        if not self.position:  
            if self.crossover > 0:  
                size = (self.broker.getcash() * self.p.risk_perc) / self.data.close[0]
                self.buy(size=size)  
        elif self.crossover < 0:  
            self.close()  

def fetch_crypto_data(symbol='BTC/USDT', timeframe='4h', start_date='2024-01-01', limit=1000):
    exchange = ccxt.binance()
    
    from_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
    
    # Fetch data in chunks
    all_ohlcv = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, from_timestamp, limit=limit)
        if not ohlcv:  # No more data
            break
            
        all_ohlcv.extend(ohlcv)
        from_timestamp = ohlcv[-1][0] + 1  # Next timestamp after last received
    
    df = pd.DataFrame(all_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    df.set_index('datetime', inplace=True)
    
    print(f"\nBacktest Period:")
    print(f"Start: {df.index[0].strftime('%Y-%m-%d')}")
    print(f"End: {df.index[-1].strftime('%Y-%m-%d')}")
    print(f"Total Days: {(df.index[-1] - df.index[0]).days}")
    print(f"Total Candles: {len(df)}")
    
    return df

cerebro = bt.Cerebro()

# Fixed end date for consistency
df = fetch_crypto_data(symbol='LINK/USDT', timeframe='4h', start_date='2024-01-01', limit=1000)

data = bt.feeds.PandasData(
    dataname=df,
    datetime=None,
    open=0,
    high=1,
    low=2,
    close=3,
    volume=4,
    openinterest=-1
)

cerebro.adddata(data)
cerebro.addstrategy(SmaCross)

starting_cash = 100000
cerebro.broker.setcash(starting_cash)
cerebro.broker.setcommission(commission=0.001)

cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

results = cerebro.run()
strategy = results[0]

returns = strategy.analyzers.returns.get_analysis()
trades = strategy.analyzers.trades.get_analysis()

print('\nBacktest Results:')
print(f'Starting Portfolio Value: ${starting_cash:,.2f}')
print(f'Final Portfolio Value: ${cerebro.broker.getvalue():,.2f}')
print(f'Net Profit: ${cerebro.broker.getvalue() - starting_cash:,.2f}')
print(f'Return: {returns["rtot"]:.2%}')
print(f'Max Drawdown: {strategy.max_drawdown:.2%}')
print(f'Number of Trades: {trades.total.total}')
print(f'Winning Trades: {trades.won.total}')
print(f'Losing Trades: {trades.lost.total}')


Backtest Period:
Start: 2024-01-01
End: 2024-11-04
Total Days: 308
Total Candles: 1851

Backtest Results:
Starting Portfolio Value: $100,000.00
Final Portfolio Value: $121,857.21
Net Profit: $21,857.21
Return: 19.77%
Max Drawdown: 32.31%
Number of Trades: 34
Winning Trades: 16
Losing Trades: 18


### Basic SMA crossover strategy with trades log

In [4]:
import backtrader as bt
import pandas as pd
import ccxt
from datetime import datetime
import numpy as np

class SmaCross(bt.Strategy):
    params = dict(
        pfast=10,  
        pslow=30,  
        risk_perc=0.95  
    )
    def __init__(self):
        self.sma1 = bt.ind.SMA(self.data.close, period=self.p.pfast)
        self.sma2 = bt.ind.SMA(self.data.close, period=self.p.pslow)
        self.crossover = bt.ind.CrossOver(self.sma1, self.sma2)
        
        # Track performance metrics
        self.start_value = self.broker.getvalue()
        self.max_value = self.start_value
        self.max_drawdown = 0
        self.total_return = 0
        self.cagr = 0
        self.start_date = None
        
        # Track trades and returns
        self.trade_list = []
        self.returns = []

    def notify_trade(self, trade):
        if trade.isclosed:
            entry_price = trade.price
            exit_price = self.data.close[0]  # Current closing price
            trade_return = (exit_price - entry_price) / entry_price if entry_price != 0 else 0
            
            self.trade_list.append({
                'entry_date': self.data.datetime.datetime(-trade.barlen),
                'exit_date': self.data.datetime.datetime(0),
                'duration': trade.barlen,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'pnl': trade.pnl,
                'return': trade_return
            })
            self.returns.append(trade_return)

    def next(self):
        # Update dates and values
        if self.start_date is None:
            self.start_date = self.data.datetime.datetime(0)
        
        current_value = self.broker.getvalue()
        current_date = self.data.datetime.datetime(0)
        
        # Update metrics on each candle
        self.max_value = max(self.max_value, current_value)
        self.total_return = (current_value - self.start_value) / self.start_value
        
        # Update drawdown
        drawdown = (self.max_value - current_value) / self.max_value
        self.max_drawdown = max(self.max_drawdown, drawdown)
        
        # Update CAGR
        years = (current_date - self.start_date).days / 365.25
        if years > 0:  # Avoid division by zero
            self.cagr = (current_value / self.start_value) ** (1/years) - 1
        
        # Trading logic
        if not self.position:  
            if self.crossover > 0:  
                size = (self.broker.getcash() * self.p.risk_perc) / self.data.close[0]
                self.buy(size=size)  
        elif self.crossover < 0:  
            self.close()

def fetch_crypto_data(symbol='BTC/USDT', timeframe='4h', start_date='2024-01-01', limit=1000):
    exchange = ccxt.binance()
    
    from_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
    
    all_ohlcv = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, from_timestamp, limit=limit)
        if not ohlcv:  
            break
            
        all_ohlcv.extend(ohlcv)
        from_timestamp = ohlcv[-1][0] + 1
    
    df = pd.DataFrame(all_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    df.set_index('datetime', inplace=True)
    
    print(f"\nBacktest Period:")
    print(f"Start: {df.index[0].strftime('%Y-%m-%d')}")
    print(f"End: {df.index[-1].strftime('%Y-%m-%d')}")
    print(f"Total Days: {(df.index[-1] - df.index[0]).days}")
    print(f"Total Candles: {len(df)}")
    
    return df

cerebro = bt.Cerebro()
df = fetch_crypto_data(symbol='BTC/USDT', timeframe='4h', start_date='2018-04-01', limit=10000)
data = bt.feeds.PandasData(
    dataname=df,
    datetime=None,
    open=0,
    high=1,
    low=2,
    close=3,
    volume=4,
    openinterest=-1
)

cerebro.adddata(data)
cerebro.addstrategy(SmaCross)
starting_cash = 100000
cerebro.broker.setcash(starting_cash)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

results = cerebro.run()
strategy = results[0]
trades = strategy.analyzers.trades.get_analysis()

print('\nBacktest Results:')
print(f'Starting Portfolio Value: ${starting_cash:,.2f}')
print(f'Final Portfolio Value: ${cerebro.broker.getvalue():,.2f}')
print(f'Total Return: {strategy.total_return:.2%}')
print(f'CAGR: {strategy.cagr:.2%}')
print(f'Max Drawdown: {strategy.max_drawdown:.2%}')
print(f'Number of Trades: {trades.total.total}')
print(f'Winning Trades: {trades.won.total}')
print(f'Losing Trades: {trades.lost.total}')

# Print trade statistics
if len(strategy.returns) > 0:
    returns_array = np.array(strategy.returns)
    print('\nTrade Statistics:')
    print(f'Average Trade Return: {np.mean(returns_array):.2%}')
    print(f'Trade Return Std Dev: {np.std(returns_array):.2%}')
    print(f'Best Trade: {np.max(returns_array):.2%}')
    print(f'Worst Trade: {np.min(returns_array):.2%}')

print('\nDetailed Trades:')
for i, trade in enumerate(strategy.trade_list, 1):
    print(f"\nTrade {i}:")
    print(f"Entry Date: {trade['entry_date']}")
    print(f"Exit Date: {trade['exit_date']}")
    print(f"Duration: {trade['duration']} bars")
    print(f"Entry Price: ${trade['entry_price']:.2f}")
    print(f"Exit Price: ${trade['exit_price']:.2f}")
    print(f"P&L: ${trade['pnl']:.2f}")
    print(f"Return: {trade['return']:.2%}")


Backtest Period:
Start: 2018-04-01
End: 2024-11-04
Total Days: 2409
Total Candles: 14449

Backtest Results:
Starting Portfolio Value: $100,000.00
Final Portfolio Value: $674,753.00
Total Return: 574.75%
CAGR: 33.65%
Max Drawdown: 67.19%
Number of Trades: 278
Winning Trades: 94
Losing Trades: 184

Trade Statistics:
Average Trade Return: 1.01%
Trade Return Std Dev: 8.46%
Best Trade: 47.63%
Worst Trade: -12.83%

Detailed Trades:

Trade 1:
Entry Date: 2018-04-08 12:00:00
Exit Date: 2018-04-10 16:00:00
Duration: 13 bars
Entry Price: $7100.00
Exit Price: $6844.93
P&L: $-3537.15
Return: -3.59%

Trade 2:
Entry Date: 2018-04-12 12:00:00
Exit Date: 2018-04-17 20:00:00
Duration: 32 bars
Entry Price: $7685.55
Exit Price: $7885.02
P&L: $2241.37
Return: 2.60%

Trade 3:
Entry Date: 2018-04-19 16:00:00
Exit Date: 2018-04-26 16:00:00
Duration: 42 bars
Entry Price: $8235.45
Exit Price: $8881.54
P&L: $6470.17
Return: 7.85%

Trade 4:
Entry Date: 2018-04-28 04:00:00
Exit Date: 2018-05-01 12:00:00
Duration

### Confirmed SMA crossover Long-Short strategy with trades log

In [4]:
import backtrader as bt
import pandas as pd
import ccxt
from datetime import datetime
import numpy as np

class SmaCross(bt.Strategy):
   params = dict(
       stf_pfast=10,  
       stf_pslow=30, 
       ltf_pfast=20,  
       ltf_pslow=50, 
       risk_perc=0.99,
       stop_loss_perc=0.02
   )
   def __init__(self):
       
       # Initialize variables for tracking entry prices and stop losses
       self.entry_price = None
       self.stop_loss_price = None
       
       # Get both timeframes (STF = Short TimeFrame, LTF = Long TimeFrame)
       self.data_stf = self.datas[0]
       self.data_ltf = self.datas[1]
       
       # Calculate SMAs for both timeframes
       self.sma1_stf = bt.ind.SMA(self.data_stf.close, period=self.p.stf_pfast)
       self.sma2_stf = bt.ind.SMA(self.data_stf.close, period=self.p.stf_pslow)
       self.crossover_stf = bt.ind.CrossOver(self.sma1_stf, self.sma2_stf)
       
       self.sma1_ltf = bt.ind.SMA(self.data_ltf.close, period=self.p.ltf_pfast)
       self.sma2_ltf = bt.ind.SMA(self.data_ltf.close, period=self.p.ltf_pslow)
       
       # Volume indicators
       self.vol_sma = bt.ind.SMA(self.data_stf.volume, period=20)
       self.vol_std = bt.ind.StdDev(self.data_stf.volume, period=20)
       self.price_change = self.data_stf.close - self.data_stf.open
       self.vol_delta = (self.data_stf.volume - self.vol_sma) / self.vol_std

       #Initialize trade size 
       self.trade_size = None  

       
       # Track performance metrics
       self.start_value = self.broker.getvalue()
       self.max_value = self.start_value
       self.max_drawdown = 0
       self.total_return = 0
       self.ret_mdd_ratio = 0
       self.cagr = 0
       self.start_date = None
       
       # Track trades and returns
       self.trade_list = []
       self.returns = []

   def volume_confirmation(self):
       # Current volume metrics
       current_volume = self.data_stf.volume[0]
       avg_volume = self.vol_sma[0]
       vol_delta = self.vol_delta[0]
       price_change = self.price_change[0]
       
       # Volume conditions
       above_average = current_volume > avg_volume * 1.2  # 10% above average
       strong_momentum = vol_delta > 0.2  # More than 1 std dev above mean
       price_aligned = (price_change > 0 and self.crossover_stf > 0) or \
                      (price_change < 0 and self.crossover_stf < 0)
       
       return above_average and strong_momentum and price_aligned
       
   def notify_trade(self, trade):
        if trade.isclosed:
            entry_price = trade.price
            exit_price = self.data_stf.close[0]
            size = self.trade_size
            
            # Determine if the trade was long or short based on trade.size
            position_type = 'Long' if size > 0 else 'Short'
            
            # Calculate PnL and return based on trade direction
            if size >= 0:  # Long position
                trade_return = (exit_price - entry_price) / entry_price if entry_price != 0 else 0
                pnl = (exit_price - entry_price) * abs(size)
            else:  # Short position
                trade_return = (entry_price - exit_price) / entry_price if entry_price != 0 else 0
                pnl = (entry_price - exit_price) * abs(size)
            
            # Store trade information
            self.trade_list.append({
                'position_type': position_type,
                'trade_size': size,
                'entry_date': self.data_stf.datetime.datetime(-trade.barlen),
                'exit_date': self.data_stf.datetime.datetime(0),
                'duration': trade.barlen,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'pnl': pnl,
                'return': trade_return
            })
            
            # Track returns
            self.returns.append(trade_return)

            # Reset the trade size
            self.trade_size = None


   def next(self):
       # Update dates and values
       if self.start_date is None:
           self.start_date = self.data_stf.datetime.datetime(0)
       
       current_value = self.broker.getvalue()
       current_date = self.data_stf.datetime.datetime(0)
       
       # Update metrics on each candle
       self.max_value = max(self.max_value, current_value)
       self.total_return = (current_value - self.start_value) / self.start_value
       
       # Update drawdown
       drawdown = (self.max_value - current_value) / self.max_value
       self.max_drawdown = max(self.max_drawdown, drawdown)

       # Update risk-adjusted-return
       if self.max_drawdown != 0:
           self.ret_mdd_ratio = self.total_return / self.max_drawdown
       else:
           self.ret_mdd_ratio = 0
       
       # Update CAGR
       years = (current_date - self.start_date).days / 365.25
       if years > 0:
           self.cagr = (current_value / self.start_value) ** (1/years) - 1
       
       # Check if we have data for both timeframes
       if len(self.data_ltf) > 0:
           current_price = self.data_stf.close[0]
           vol_confirmed = self.volume_confirmation()
           
           # Trading logic
           if not self.position:
               # Long signal: STF crossover and LTF trend up
               if (self.crossover_stf > 0 and 
                   self.sma1_ltf > self.sma2_ltf and 
                   current_price > self.sma1_ltf and
                   vol_confirmed):
                    size = (self.broker.getcash() * self.p.risk_perc) / self.data_stf.close[0]
                    self.trade_size = size
                    self.buy(size=size)
                    self.entry_price = current_price
                    self.stop_loss_price = self.entry_price * (1 - self.p.stop_loss_perc)  # Set stop loss

               # Short signal: STF crossover and LTF trend down
               elif (self.crossover_stf < 0 and 
                     self.sma1_ltf < self.sma2_ltf and 
                     current_price < self.sma1_ltf and
                     vol_confirmed):
                   size = (self.broker.getcash() * self.p.risk_perc) / self.data_stf.close[0]
                   self.trade_size = -size
                   self.sell(size=size)
                   self.entry_price = current_price
                   self.stop_loss_price = self.entry_price * (1 + self.p.stop_loss_perc)  # Set stop loss for short

           else:
               # Close long if STF crosses down or LTF trend changes
               if self.position.size > 0:
                   if current_price <= self.stop_loss_price or (self.crossover_stf < 0 or self.sma1_ltf < self.sma2_ltf or current_price < self.sma1_ltf):
                       self.close()

               # Close short if STF crosses up or LTF trend changes
               elif self.position.size < 0:
                   if current_price >= self.stop_loss_price or (self.crossover_stf > 0 or self.sma1_ltf > self.sma2_ltf or current_price > self.sma1_ltf):
                       self.close()

def fetch_crypto_data(symbol, timeframe, start_date, limit):
    exchange = ccxt.binance()
    
    from_timestamp = int(pd.Timestamp(start_date).timestamp() * 1000)
    to_timestamp = int(pd.Timestamp.now().timestamp() * 1000)
    
    all_ohlcv = []
    while True:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, from_timestamp, limit=limit)
        if not ohlcv or from_timestamp >= to_timestamp:
            break
            
        all_ohlcv.extend(ohlcv)
        from_timestamp = ohlcv[-1][0] + 1
    
    df = pd.DataFrame(all_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    df.set_index('datetime', inplace=True)
    
    return df

# Fetch data and align timeframes
cerebro = bt.Cerebro()
start_date = '2016-09-01'

# Get both timeframes
df_stf = fetch_crypto_data(symbol='BTC/USDT', timeframe='4h', start_date=start_date, limit=10000)
df_ltf = fetch_crypto_data(symbol='BTC/USDT', timeframe='1w', start_date=start_date, limit=10000)

# Align the periods
common_start = max(df_stf.index[0], df_ltf.index[0])
common_end = min(df_stf.index[-1], df_ltf.index[-1])

df_stf = df_stf[common_start:common_end]
df_ltf = df_ltf[common_start:common_end]

# Print aligned periods
print("\nAligned Backtest Period:")
print(f"Start: {common_start.strftime('%Y-%m-%d')}")
print(f"End: {common_end.strftime('%Y-%m-%d')}")
print(f"Total Days: {(common_end - common_start).days}")
print(f"\nSTF Candles: {len(df_stf)}")
print(f"LTF Candles: {len(df_ltf)}")

# Add both timeframes to cerebro
data_stf = bt.feeds.PandasData(
   dataname=df_stf,
   datetime=None,
   open=0,
   high=1,
   low=2,
   close=3,
   volume=4,
   openinterest=-1
)

data_ltf = bt.feeds.PandasData(
   dataname=df_ltf,
   datetime=None,
   open=0,
   high=1,
   low=2,
   close=3,
   volume=4,
   openinterest=-1
)

cerebro.adddata(data_stf)  
cerebro.adddata(data_ltf) 

cerebro.addstrategy(SmaCross)
starting_cash = 100000
cerebro.broker.setcash(starting_cash)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

results = cerebro.run()
strategy = results[0]
trades = strategy.analyzers.trades.get_analysis()

print('\nBacktest Results:')
print(f'Starting Portfolio Value: ${starting_cash:,.2f}')
print(f'Final Portfolio Value: ${cerebro.broker.getvalue():,.2f}')
print(f'Total Return: {strategy.total_return:.2%}')
print(f'CAGR: {strategy.cagr:.2%}')
print(f'Max Drawdown: {strategy.max_drawdown:.2%}')
print(f'Risk-adjusted return: {strategy.ret_mdd_ratio:.2f}')
print(f'Number of Trades: {trades.total.total}')
print(f'Winning Trades: {trades.won.total}')
print(f'Losing Trades: {trades.lost.total}')

if len(strategy.returns) > 0:
   returns_array = np.array(strategy.returns)
   print('\nTrade Statistics:')
   print(f'Average Trade Return: {np.mean(returns_array):.2%}')
   print(f'Trade Return Std Dev: {np.std(returns_array):.2%}')
   print(f'Best Trade: {np.max(returns_array):.2%}')
   print(f'Worst Trade: {np.min(returns_array):.2%}')

print('\nDetailed Trades:')
for i, trade in enumerate(strategy.trade_list, 1):
    print(f"\nTrade {i}:")
    print(f"Position type: {trade['position_type']}")
    print(f"Entry Date: {trade['entry_date']}")
    print(f"Exit Date: {trade['exit_date']}")
    print(f"Duration: {trade['duration']} bars")
    print(f"Entry Price: ${trade['entry_price']:.2f}")
    print(f"Exit Price: ${trade['exit_price']:.2f}")
    print(f"P&L: ${trade['pnl']:.2f}")
    print(f"Return: {trade['return']:.2%}")


Aligned Backtest Period:
Start: 2017-08-17
End: 2024-11-04
Total Days: 2635

STF Candles: 15800
LTF Candles: 377

Backtest Results:
Starting Portfolio Value: $100,000.00
Final Portfolio Value: $263,856.27
Total Return: 163.86%
CAGR: 16.74%
Max Drawdown: 25.22%
Risk-adjusted return: 6.50
Number of Trades: 37
Winning Trades: 13
Losing Trades: 24

Trade Statistics:
Average Trade Return: 2.92%
Trade Return Std Dev: 11.68%
Best Trade: 39.41%
Worst Trade: -10.12%

Detailed Trades:

Trade 1:
Position type: Short
Entry Date: 2018-08-14 04:00:00
Exit Date: 2018-08-15 00:00:00
Duration: 5 bars
Entry Price: $5974.31
Exit Price: $6289.99
P&L: $-5230.56
Return: -5.28%

Trade 2:
Position type: Short
Entry Date: 2018-08-21 00:00:00
Exit Date: 2018-08-21 08:00:00
Duration: 2 bars
Entry Price: $6251.00
Exit Price: $6431.12
P&L: $-2744.17
Return: -2.88%

Trade 3:
Position type: Short
Entry Date: 2018-09-06 00:00:00
Exit Date: 2018-09-13 12:00:00
Duration: 45 bars
Entry Price: $6697.27
Exit Price: $6480

### Refactored code

In [8]:
%load_ext autoreload
%autoreload 2
    
import backtrader as bt
from utils.fetch_data import fetch_crypto_data
from utils.bt_outputs import get_bt_results
from utils.prep_data import align_data_periods, format_data_cerebro
from strategies.strategies import BuyHold, SmaSimpleCrossL, SmaConfCrossLS 

# Instantiate backtest
cerebro = bt.Cerebro()
start_date = '2019-01-01'
starting_cash = 100000

# Get both timeframes and align periods
df_stf = fetch_crypto_data(symbol='BTC/USDT', timeframe='4h', start_date=start_date, limit=10000)
df_ltf = fetch_crypto_data(symbol='BTC/USDT', timeframe='1w', start_date=start_date, limit=10000)
df_stf, df_ltf = align_data_periods(df_stf, df_ltf)
[data_stf, data_ltf] = format_data_cerebro([df_stf, df_ltf])

# Launch strategy backtest
cerebro.adddata(data_stf)
cerebro.adddata(data_ltf)
cerebro.addstrategy(SmaConfCrossLS)
cerebro.broker.setcash(starting_cash)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

results = cerebro.run()
strategy = results[0]
trades = strategy.analyzers.trades.get_analysis()

# Get strategy backtest results
get_bt_results(starting_cash, cerebro, strategy, trades)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Aligned Backtest Period:
Start: 2019-01-07
End: 2024-11-04
Total Days: 2128

STF Candles: 12764
LTF Candles: 305

Backtest Results:
Starting Portfolio Value: $100,000.00
Final Portfolio Value: $215,628.28
Total Return: 115.63%
CAGR: 17.03%
Max Drawdown: 9.82%
Risk-adjusted return: 11.78
Number of Trades: 11
Winning Trades: 4
Losing Trades: 7

Trade Statistics:
Average Trade Return: 8.37%
Trade Return Std Dev: 14.06%
Best Trade: 38.49%
Worst Trade: -4.17%

Detailed Trades:

Trade 1:
Position type: Long
Position size: 1.8320369975245294
Entry Date: 2021-04-27 00:00:00
Exit Date: 2021-04-30 00:00:00
Duration: 18 bars
Entry Price: $54001.38
Exit Price: $53555.00
P&L: $-817.78
Return: -0.83%

Trade 2:
Position type: Long
Position size: 2.251093357234109
Entry Date: 2021-10-01 00:00:00
Exit Date: 2021-10-23 00:00:00
Duration: 132 bars
Entry Price: $43820.01
Exit Price: $60688.23
P&L: $37971.94
Return: 38

In [94]:
%load_ext autoreload
%autoreload 2
    
import backtrader as bt
import pandas as pd
from utils.fetch_data import fetch_crypto_data
from utils.bt_outputs import get_bt_results
from utils.prep_data import align_data_periods, format_data_cerebro
from backtest.run_bt import generate_bootstrap_periods, run_backtest, run_stability_analysis, print_stability_stats, optimize_strategy
from strategies.strategies import BuyHold, SmaSimpleCrossL, SmaConfCrossLS, RSIBBStrategy

# Generate start dates
start_dates = generate_bootstrap_periods(start='2017-01-01', end='2024-11-01', period_length_days=1050, n_samples=20)

# Run stability analysis
results_df, stability_stats = run_stability_analysis('BTC/USDT', start_dates, RSIBBStrategy, ['4h'], period_length_days=1050)

# Print results
print_stability_stats(stability_stats)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Aligned Backtest Period:
Start: 2017-08-17
End: 2019-12-25
Total Days: 859

Candles per DataFrame: [5145]
Completed backtest for start date: 2017-02-08

Aligned Backtest Period:
Start: 2017-08-17
End: 2020-04-25
Total Days: 981

Candles per DataFrame: [5876]
Completed backtest for start date: 2017-06-10

Aligned Backtest Period:
Start: 2017-08-17
End: 2020-05-05
Total Days: 991

Candles per DataFrame: [5936]
Completed backtest for start date: 2017-06-20

Aligned Backtest Period:
Start: 2017-11-17
End: 2020-10-02
Total Days: 1050

Candles per DataFrame: [6285]
Completed backtest for start date: 2017-11-17

Aligned Backtest Period:
Start: 2017-12-17
End: 2020-11-01
Total Days: 1050

Candles per DataFrame: [6285]
Completed backtest for start date: 2017-12-17

Aligned Backtest Period:
Start: 2018-01-18
End: 2020-12-03
Total Days: 1050

Candles per DataFrame: [6285]
Completed backtest for start date: 20

In [55]:
%load_ext autoreload
%autoreload 2
    
import backtrader as bt
import pandas as pd
from utils.fetch_data import fetch_crypto_data
from utils.bt_outputs import get_bt_results
from utils.prep_data import align_data_periods, format_data_cerebro
from backtest.run_bt import run_train_test_analysis, analyze_test_results
from strategies.strategies import BuyHold, SmaSimpleCrossL, SmaConfCrossLS, RSIBBStrategy

# Define parameter ranges for various strategies

param_ranges_rsi_bb = {
    'rsi_period': range(10, 31, 5),          
    'bb_period': range(20, 41, 5),           
    'rsi_threshold_low': range(20, 41, 5),   
    'rsi_threshold_high': range(60, 81, 5),  
    'sl_coef': [round(x/10, 1) for x in range(10, 21, 2)],  
    'tp_coef': [round(x/10, 1) for x in range(10, 21, 2)]   
}

param_ranges_sma_conf = {
   'stf_pfast': range(5, 31, 5),            
   'stf_pslow': range(30, 100, 10),           
   'ltf_pfast': range(5, 21, 3),           
   'ltf_pslow': range(30, 51, 5),          
   'last_to_avg_volume_ratio': [i/10 for i in range(10, 16)],  
   'vol_delta_lb': [i/10 for i in range(0, 16, 2)],  
   'sl_coef': [i/10 for i in range(10, 21, 2)],      
   'tp_coef': [i/10 for i in range(10, 21, 2)]       
}

# Run train-test analysis
df_results, stats = run_train_test_analysis(
   symbol='BTC/USDT',
   start_date='2017-01-01',
   end_date='2024-10-31',
   strategy_class=SmaConfCrossLS,
   timeframes=['4h','1w'],              
   train_length_days=600,          
   test_length_days=600,            
   param_ranges=param_ranges_sma_conf,
   n_samples=3,                   # Number of bootstrap samples
   n_trials=3,
   gap_days=7,                     # 1 week gap between train and test
   starting_cash=100000,
   commission=0.001,
   limit=10000
)

best_params = analyze_test_results(df_results)
print(best_params)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Global period: 2017-01-01 to 2024-10-31
Train period sample: 2018-12-16 to 2020-08-07
Test period sample:  2020-08-14 to 2022-04-06

Trial 1/3...
Start date: 2018-12-17 00:00:00
End date: 2020-08-03 00:00:00
Duration days: 595
Total return: 0.00%
CAGR: 0.00%
MDD: 0.00%
CAGR/MDD: 0.00
Error in trial 1: 

Trial 2/3...
Start date: 2018-12-17 00:00:00
End date: 2020-08-03 00:00:00
Duration days: 595
Total return: 0.00%
CAGR: 0.00%
MDD: 0.00%
CAGR/MDD: 0.00
Error in trial 2: 

Trial 3/3...
Start date: 2018-12-17 00:00:00
End date: 2020-08-03 00:00:00
Duration days: 595
Total return: 0.00%
CAGR: 0.00%
MDD: 0.00%
CAGR/MDD: 0.00
Error in trial 3: 
Debug info:
Parameter ranges: {'stf_pfast': range(5, 31, 5), 'stf_pslow': range(30, 100, 10), 'ltf_pfast': range(5, 21, 3), 'ltf_pslow': range(30, 51, 5), 'last_to_avg_volume_ratio': [1.0, 1.1, 1.2, 1.3, 1.4, 1.5], 'vol_delta_lb': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1